In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow import keras

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

tf.keras.backend.clear_session() 

In [8]:
train_data = pd.read_csv("Data/train.csv")

target = train_data['label']
train_vars = train_data.drop(['label'],axis=1)

X_train = train_vars/255
y = target

X_train = X_train.values.reshape(X_train.shape[0],28,28,1)

In [9]:
inputs = keras.Input(shape=(28, 28, 1), name='img')
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.BatchNormalization()(x)
block_1_output = layers.MaxPooling2D(3)(x)

x = layers.Conv2D(64, 3, activation='relu', padding='same')(block_1_output)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
block_2_output = layers.add([x, block_1_output])

x = layers.Conv2D(64, 3, activation='relu', padding='same')(block_2_output)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
block_3_output = layers.add([x, block_2_output])

x = layers.Conv2D(64, 3, activation='relu', padding='same')(block_3_output)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
block_4_output = layers.add([x, block_3_output])



x = layers.Conv2D(64, 3, activation='relu')(block_4_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs, name='kannada_resnet')
model.summary()


Model: "kannada_resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 26, 26, 32)   320         img[0][0]                        
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 26, 26, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 24, 24, 64)   18496       batch_normalization[0][0]        
_____________________________________________________________________________________

In [11]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=8)
lr_reduction = ReduceLROnPlateau(monitor='val_loss',patience=5,verbose=1,factor=0.2)

In [13]:
model.fit(X_train,np.array(y),
          epochs=50,validation_split=0.2,
         batch_size=128, shuffle=True,callbacks =[lr_reduction,es])

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 105s 2ms/sample - loss: 0.1741 - accuracy: 0.9447 - val_loss: 3.6545 - val_accuracy: 0.3485
Epoch 2/50
48000/48000 [==============================] - 108s 2ms/sample - loss: 0.0335 - accuracy: 0.9910 - val_loss: 0.0302 - val_accuracy: 0.9918
Epoch 3/50
48000/48000 [==============================] - 110s 2ms/sample - loss: 0.0227 - accuracy: 0.9940 - val_loss: 0.0423 - val_accuracy: 0.9899
Epoch 4/50
48000/48000 [==============================] - 108s 2ms/sample - loss: 0.0186 - accuracy: 0.9949 - val_loss: 0.0290 - val_accuracy: 0.9918
Epoch 5/50
48000/48000 [==============================] - 107s 2ms/sample - loss: 0.0156 - accuracy: 0.9956 - val_loss: 0.0198 - val_accuracy: 0.9940
Epoch 6/50
48000/48000 [==============================] - 107s 2ms/sample - loss: 0.0142 - accuracy: 0.9958 - val_loss: 0.0215 - val_accuracy: 0.9942
Epoch 7/50
48000/48000 [==========================

In [17]:
test_data = pd.read_csv('Data/test.csv')
ImageId = test_data['id']
NN_test = test_data.drop(['id'],axis=1)
NN_test = NN_test/255
NN_test = NN_test.values.reshape(NN_test.shape[0],28,28,1)

predictions = np.argmax(model.predict(NN_test),axis=1)

In [20]:
sub = pd.DataFrame({'id':ImageId, 'label':predictions})
sub.to_csv("submission.csv",index=False)